In [21]:
from openai import OpenAI
import base64
import time

In [27]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
def analyze_image(test_image, fsp_img1):

    api_key = ''
    client = OpenAI(api_key=api_key)
    model = "gpt-4o"

    questions = '''
    For each question, after you answer, add any assumptions you are making:

    Which objects in the frame can be used as a weapon?
    Is there a crowd of people in the frame?
    How many / which objects in the frame are intended for safety?
    How many / which communication devices (phones, radios, etc.) are available?
    How many / which transportation mediums are available (car, bus, bike, etc.)?
    Is there adequate lighting to navigate safely?
    Estimate the total weight of any heavy objects (>1000 lbs) in the photo.
    '''

    start_time = time.time()

    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content":
            '''
            You are a safety officer reporting on the quality of safety and potential threats in an image. 
            Your job is to identify objects in the image that both directly and indirectly relate to potential threats and safety. 
            Focus on objects like potential weapons, communication devices, people, and transportation, as well as objects that could have an underlying purpose of safety. 
            Assess the purpose of each object in the image and answer the following questions while keeping these considerations in mind.
            '''
        },

        {"role": "user", "content": [
            
            # Few Shot Prompting
            {
            "type": "text",
                "text":
                questions
            },
            {
            "type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{encode_image(fsp_img1)}"}                        
            },
            {
            "type": "text",
                "text": 
                '''
                1. The police officers are carrying batons or sticks that can be used as blunt force weapons. I am making the assumption that weapons are objects designed or intended to cause harm or injury.
                2. Yes, there is a crowd of over 20 police officers in the center of this image. I am making the assumption that a crowd exists if 3 or more humans are within 5 feet of each other.
                3. The police officers are all wearing safety vests and helmets to protect them from threats. The police officers are also jumping over a barricade which is used to safely keep people in and out of certain areas. In the background, you can also see traffic lights used to control traffic safely. I am making the assumption that safety objects are anything that can be used to help control or provide safety to participants in the image.
                4. There are no communication devices in the image. I am making the assumption that communication devices include electronic devices like phones, radios, or computers or non-electronic methods of communication.
                5. There are no mediums of transportation visible, but there are likely some nearby as the image is set on the street. I am making the assumption that transportation mediums include anything that can move you from place to place.
                6. Although it is dusk, there appears to be adequate lighting due to the bright sky and numerous street lights in the background. I am making the assumption that a bright sky indicates sufficient ambient lighting and external light sources also help.
                7. The concrete barrier weighs tens of thousands of pounds. I am making the assumption that each segment of the concrete barricade weighs roughly 1500lbs.

                '''
            },

            # {
            # "type": "text",
            #     "text":
            #     questions
            # },
            # {
            # "type": "image_url", "image_url": {
            #     "url": f"data:image/png;base64,{encode_image(fsp_img2)}"}                        
            # },
            # {
            # "type": "text",
            #     "text": 
            #     '''


            #     '''
            # },

            # Test Image
            {"type": "text", "text": 
             questions + 
             "Use the format and type of the response in the previous prompts to answer these questions for the following image."
            },

            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{encode_image(test_image)}"}
            }
        ]}
    ],
    temperature=0.7,
    )

    end_time = time.time()

    answer = response.choices[0].message.content
    runtime = round(end_time - start_time, 2)

    return answer, runtime


# You are an AI assistant that helps increase situational awareness by assessing potential threats and safety considerations based on an image of an environment. Only describe what is visible in the image.

# You are a police officer trying to keep your city safe. Your job is to identify objects in the image that directly relate to potential threats and safety. Focus on objects like potential weapons, people, transportation, and heavy things. Answer the following questions while keeping these considerations in mind.

# You are a safety officer reporting on the quality of safety and potential threats in an image. Your job is to identify objects in the image that both directly and indirectly relate to potential threats and safety. 
# Focus on objects like potential weapons, people, and transportation, as well as objects that could have an underlying purpose of safety. Answer the following questions while keeping these considerations in mind.



In [29]:
# IMAGE_PATH = 'data/train.jpg'
answer, runtime = analyze_image('data/train.jpg', 'data/riot.png')
print(answer)
print(runtime)

1. The backpacks and bags could potentially be used as blunt force weapons. I am assuming that any object that can be swung or thrown to cause harm may be considered a weapon.

2. Yes, there is a small group of three people in the frame. I am assuming a crowd can be any group of three or more people close together.

3. There are no specific objects intended for safety visible in the image. I am assuming safety objects include items like helmets, safety vests, or barriers.

4. One person appears to be holding a phone, which is a communication device. I am assuming communication devices include electronic devices like phones and radios.

5. A train is visible in the background, which is a medium of transportation. I am assuming transportation mediums include vehicles like cars, buses, trains, and bikes.

6. There appears to be adequate lighting, as it is daytime and the area is well-lit. I am assuming daylight and visible light sources indicate sufficient lighting.

7. The train is a hea

In [10]:
answer, runtime = analyze_image('data/street.jpg', 'data/riot.png')
print(answer)
print(runtime)

1. **Which objects in the frame can be used as a weapon?**
   - The objects that could be used as weapons include bicycles and potentially any hard objects like the vehicles themselves if used with harmful intent. I assume weapons are any objects that can cause harm.

2. **Is there a crowd of people in the frame?**
   - Yes, there is a crowd of people walking along the sidewalks. I assume a crowd exists if 3 or more people are within close proximity.

3. **How many / which objects in the frame are intended for safety?**
   - The streetlights and road markings are intended for safety, helping to manage traffic flow. I assume safety objects are those that help prevent accidents or injuries.

4. **How many / which communication devices (phones, radios, etc.) are available?**
   - There are no visible communication devices in the image. I assume communication devices include electronic devices like phones or radios.

5. **How many / which transportation mediums are available (car, bus, bik

In [28]:
answer, runtime = analyze_image('data/8454124021_48393547a6_z.jpg', 'data/riot.png')
print(answer)
print(runtime)

1. **Which objects in the frame can be used as a weapon?**

   The kitchen contains knives and other utensils that could be used as blunt force weapons. 

   *Assumption: Kitchen tools such as knives and heavy utensils can be used as weapons.*

2. **Is there a crowd of people in the frame?**

   No, there are only a few chefs visible. 

   *Assumption: A crowd is defined as more than three people in close proximity.*

3. **How many / which objects in the frame are intended for safety?**

   There are no specific safety objects visible, such as fire extinguishers or first aid kits.

   *Assumption: Safety objects are those explicitly designed for emergency or protective use.*

4. **How many / which communication devices (phones, radios, etc.) are available?**

   There are no visible communication devices in the image.

   *Assumption: Communication devices are electronic items like phones or radios.*

5. **How many / which transportation mediums are available (car, bus, bike, etc.)?**
